In [ ]:
import pandas as pd
import numpy as np

from bokeh.io import output_file, output_notebook,curdoc  
from bokeh.plotting import figure, show
from bokeh.models import ColumnDataSource,HoverTool, CustomJS, Select,RangeSlider
from bokeh.layouts import row, column, gridplot, widgetbox
from bokeh.models.widgets import Tabs, Panel
from datetime import date

output_file('filename.html')  
fig = figure()
df_cov = pd.read_json("https://apicovid19indonesia-v2.vercel.app/api/indonesia/harian")
df_cov.to_csv("covid_harian.csv",index=False)
df_use = df_cov[["positif","sembuh","tanggal"]]
df_use['tanggal'] = df_use['tanggal'].str[:10]

source_df = ColumnDataSource(data=df_use)
# Output the visualization directly in the notebook
output_notebook()

# Create a figure plot
fig = figure(title='Covid',
             plot_height=400, plot_width=len(df_use),
             x_axis_label='Day Number', y_axis_label='Total',
             x_minor_ticks=2, y_range=(0, df_cov["positif"].max()*1.05),
             x_range=(0, len(df_use)-1),
             tools="box_zoom,reset,save")


fig.legend.location = 'top_left'

#tooltip hover
tooltips = [('Day', '@index'),
            ('Date', '@tanggal'),
            ('Sembuh', '@sembuh'),
            ('Positif', '@positif'),
           ]


#dropdown interactive

select = Select(title="Show:", value="Line 1", options=["All", "Positif", "Sembuh"])
select.js_on_change("value", CustomJS(args=dict(line_1=fig.vbar("index", 0,"positif",  source=source_df,color='red', width=3.75,legend_label='Positif'),
                                                line_2=fig.line("index", "sembuh", source=source_df, color='green', line_width=1, legend_label='Sembuh'),
                                                line_3=fig.line("index", "positif", source=source_df, color='red', line_width=1)), code="""

line_1.visible = true
line_2.visible = true
line_3.visible = true

if (this.value === "Positif") {
    line_2.visible = false 
} else if (this.value === "Sembuh"){
    line_1.visible = false
    line_3.visible = false
}else{
  line_1.visible = true
  line_2.visible = true
  line_3.visible = true
}
    
"""))


#slider day interactive
range_slider = RangeSlider(
    title="Day Range :",
    start=0,
    end=len(df_use),
    step=1,
    value=(fig.x_range.start, fig.x_range.end),
)
  
range_slider.js_link("value", fig.x_range, "start", attr_selector=0)
range_slider.js_link("value", fig.x_range, "end", attr_selector=1)
  

fig.add_tools( HoverTool(tooltips=tooltips,mode='vline' ))


layout = column(fig,select,range_slider)
print("Tugas Besar Visualisasi Data IF-42-GAB01  
 M Fadhil 	 	  	  	 	 1301180263 
 Fairuz Abid Insanullah 	 1301180151")show(layout)